In [34]:
import pandas as pd
import numpy as np
from scipy import stats

# Создаем датафрейм df с данными
data = {
    'userid': range(1, 1001),
    'show': np.random.choice([0, 1], 1000, p=[0.5, 0.5]),
    'click': np.random.choice([0, 1], 1000, p=[0.7, 0.3]),
    'dt': ['2022-01-01']*1000
}

df = pd.DataFrame(data)

# Рассчитываем объем выборки для сплита 50/50
sample_size = len(df) // 2

# Случайно выбираем данные для контрольной и тестовой групп
control_group = df.sample(n=sample_size, replace=False)
test_group = df.drop(control_group.index)

# Рассчитываем CTR для контрольной и тестовой групп
ctr_control = control_group['click'].mean() / control_group['show'].mean()
ctr_test = test_group['click'].mean() / test_group['show'].mean()

# Выполняем t-тест для сравнения CTR
t_stat, p_value = stats.ttest_ind(control_group['click'] / control_group['show'], test_group['click'] / test_group['show'])

# Выводим результаты теста
print(f"Объем выборки для сплита 50/50: {sample_size}")
print(f"CTR для контрольной группы: {ctr_control}")
print(f"CTR для тестовой группы: {ctr_test}")
print(f"t-статистика: {t_stat}")
print(f"p-значение: {p_value}")

# Интерпретируем результаты теста
if p_value < 0.05:
    print("Различие в CTR статистически значимо")
else:
    print("Различие в CTR не является статистически значимым")


Объем выборки для сплита 50/50: 500
CTR для контрольной группы: 0.6086956521739131
CTR для тестовой группы: 0.6141078838174274
t-статистика: nan
p-значение: nan
Различие в CTR не является статистически значимым


In [36]:
import pandas as pd
import numpy as np
from scipy import stats

# Создаем датафрейм df с данными
data = {
    'userid': range(1, 1001),
    'show': np.random.choice([0, 1], 1000, p=[0.5, 0.5]),
    'click': np.random.choice([0, 1], 1000, p=[0.7, 0.3]),
    'dt': ['2022-01-01']*1000
}
df = pd.DataFrame(data)

In [42]:
import pandas as pd
import numpy as np
from scipy import stats
import random
import datetime

# Устанавливаем количество данных
n_users = 1000

# Создаем случайные данные для датафрейма
data = {
    'userid': range(1, n_users + 1),
    'show': np.random.choice([0, 1], n_users, p=[0.5, 0.5]),
    'click': np.zeros(n_users),  # устанавливаем клики по умолчанию как 0
    'dt': [datetime.datetime(2022, random.randint(1, 12), random.randint(1, 28)) for _ in range(n_users)]
}

# Устанавливаем клики только там, где были показы
data['click'] = np.where(data['show'] == 1, np.random.choice([0, 1], n_users, p=[0.7, 0.3]), 0)

df = pd.DataFrame(data)

# Рассчитываем объем выборки для сплита 50/50
sample_size = len(df) // 2

# Случайно выбираем данные для контрольной и тестовой групп
control_group = df.sample(n=sample_size, replace=False)
test_group = df.drop(control_group.index)

# Рассчитываем CTR для контрольной и тестовой групп
ctr_control = control_group['click'].mean() / control_group['show'].mean()
ctr_test = test_group['click'].mean() / test_group['show'].mean()

# Выполняем t-тест для сравнения CTR
t_stat, p_value = stats.ttest_ind(control_group['click'] / control_group['show'], test_group['click'] / test_group['show'])

# Выводим результаты теста
print(f"Объем выборки для сплита 50/50: {sample_size}")
print(f"CTR для контрольной группы: {ctr_control}")
print(f"CTR для тестовой группы: {ctr_test}")
print(f"t-статистика: {t_stat}")
print(f"p-значение: {p_value}")

# Интерпретируем результаты теста
if p_value < 0.05:
    print("Различие в CTR статистически значимо")
else:
    print("Различие в CTR не является статистически значимым")


Объем выборки для сплита 50/50: 500
CTR для контрольной группы: 0.29694323144104806
CTR для тестовой группы: 0.3359683794466403
t-статистика: nan
p-значение: nan
Различие в CTR не является статистически значимым


In [74]:
#Загружаем необходимые библиотеки
import pandas as pd
import numpy as np
from scipy import stats
import random
import datetime

# Устанавливаем количество данных
n_users = 1000

# Создаем случайные данные для датафрейма
data = {
    'userid': range(1, n_users + 1),
    'show': np.random.choice([0, 1], n_users, p=[0.5, 0.5]),
    'click': np.zeros(n_users),  # устанавливаем клики по умолчанию как 0
    'dt': [datetime.datetime(2022, random.randint(1, 12), random.randint(1, 28)) for _ in range(n_users)]
}

# Устанавливаем клики только там, где были показы
data['click'] = np.where(data['show'] == 1, np.random.choice([0, 1], n_users, p=[0.7, 0.3]), 0)

df = pd.DataFrame(data)

# Рассчитываем объем выборки для сплита 50/50
sample_size = len(df) // 2

# Случайно выбираем данные для контрольной и тестовой групп
control_group = df.sample(n=sample_size, replace=False)
test_group = df.drop(control_group.index)

# Рассчитываем CTR для контрольной и тестовой групп
ctr_control = control_group['click'].sum() / control_group['show'].sum()
ctr_test = test_group['click'].sum() / test_group['show'].sum()

# Выполняем t-тест для сравнения CTR
t_stat, p_value = stats.ttest_ind(control_group['click'] / np.maximum(control_group['show'], 1), test_group['click'] / np.maximum(test_group['show'], 1))

# Выводим результаты теста
print(f"Объем выборки для сплита 50/50: {sample_size}")
print(f"CTR для контрольной группы: {ctr_control}")
print(f"CTR для тестовой группы: {ctr_test}")
print(f"t-статистика: {t_stat}")
print(f"p-значение: {p_value}")

# Интерпретируем результаты теста
if p_value < 0.05:
    print("Различие в CTR статистически значимо")
else:
    print("Различие в CTR не является статистически значимым")


Объем выборки для сплита 50/50: 500
CTR для контрольной группы: 0.3319672131147541
CTR для тестовой группы: 0.29583333333333334
t-статистика: 0.8802663146712302
p-значение: 0.37892697555695587
Различие в CTR не является статистически значимым


In [75]:
import pandas as pd
import numpy as np
from scipy import stats
import random
import datetime

n=1000
p=0
e=0
for i in range (n):
    # Устанавливаем количество данных
    n_users = 1000

    # Создаем случайные данные для датафрейма
    data = {
        'userid': range(1, n_users + 1),
        'show': np.random.choice([0, 1], n_users, p=[0.5, 0.5]),
        'click': np.zeros(n_users),  # устанавливаем клики по умолчанию как 0
        'dt': [datetime.datetime(2022, random.randint(1, 12), random.randint(1, 28)) for _ in range(n_users)]
    }

    # Устанавливаем клики только там, где были показы
    data['click'] = np.where(data['show'] == 1, np.random.choice([0, 1], n_users, p=[0.7, 0.3]), 0)

    df = pd.DataFrame(data)

    # Рассчитываем объем выборки для сплита 50/50
    sample_size = len(df) // 2

    # Случайно выбираем данные для контрольной и тестовой групп
    control_group = df.sample(n=sample_size, replace=False)
    test_group = df.drop(control_group.index)

    # Рассчитываем CTR для контрольной и тестовой групп
    ctr_control = control_group['click'].sum() / control_group['show'].sum()
    ctr_test = test_group['click'].sum() / test_group['show'].sum()

    # Выполняем t-тест для сравнения CTR
    t_stat, p_value = stats.ttest_ind(control_group['click'] / np.maximum(control_group['show'], 1), test_group['click'] / np.maximum(test_group['show'], 1))

    # Выводим результаты теста
#     print(f"Объем выборки для сплита 50/50: {sample_size}")
#     print(f"CTR для контрольной группы: {ctr_control}")
#     print(f"CTR для тестовой группы: {ctr_test}")
#     print(f"t-статистика: {t_stat}")
#     print(f"p-значение: {p_value}")

    # Интерпретируем результаты теста
    if p_value < 0.05:
        #print("Различие в CTR статистически значимо")
        p+=1
    else:
        #print("Различие в CTR не является статистически значимым")
        e+=1

In [76]:
p/n

0.049

In [77]:
e/n

0.951

In [95]:
import math
from scipy.stats import norm

# Уровень доверия (например, 95%)
confidence_level = 0.95
# Уровень значимости (alpha)
alpha = 0.05
# Мощность теста (например, 80%)
power = 0.8

# Расчет Z-статистики на основе уровня доверия и уровня значимости
Z = norm.ppf(1 - alpha/2)

# Ожидаемая доля (для сплита 50/50)
p = 0.5
# Погрешность (размер эффекта)
effect_size = 2 * math.sqrt(p * (1 - p)) * alpha 

# Расчет объема выборки для двусторонней выборки с учетом мощности
n = math.ceil((Z**2 * 2 * p * (1-p)) / effect_size**2)
n_power = n * ((1 - power) / power)

print("Объем выборки для двусторонней выборки с учетом мощности:", math.ceil(n_power))



Объем выборки для двусторонней выборки с учетом мощности: 193


In [123]:
import math
from scipy.stats import norm

# Стандартное отклонение
sigma = 8
# Среднее в нулевой гипотезе
mu0 = 100
# Среднее в альтернативной гипотезе
mu1 = 101
# Мощность теста
power = 0.805
# Уровень значимости (alpha) для одностороннего теста
alpha = 0.05

# Расчет Z-статистики для уровня значимости alpha
Z = norm.ppf(1 - alpha/2)
print(Z)

# Расчет объема выборки для одностороннего теста
n = math.ceil(((sigma * (Z_alpha + norm.ppf(power))) / (mu1 - mu0))**2)

print("Объем выборки для достижения мощности 80% в одностороннем тесте:", n)


1.959963984540054
Объем выборки для достижения мощности 80% в одностороннем тесте: 400


In [124]:
import math
from scipy.stats import norm

# Стандартное отклонение
sigma = 8
# Среднее в нулевой гипотезе
mu0 = 100
# Среднее в альтернативной гипотезе
mu1 = 101
# Мощность теста
power = 0.805
# Уровень значимости (alpha) для двустороннего теста
alpha = 0.05

# Расчет Z-статистики для уровня значимости alpha/2 (для двустороннего теста)
Z_alpha_2 = norm.ppf(1 - alpha/2)

# Расчет объема выборки для двустороннего теста
n = math.ceil(((sigma * (Z_alpha_2 + norm.ppf(power))) / (mu1 - mu0))**2)

print("Объем выборки для достижения мощности 80% в двустороннем тесте:", n)


1.959963984540054
Объем выборки для достижения мощности 80% в двустороннем тесте: 509


In [ ]:
###################################################

In [126]:
# Данный расчет 
import math

# Заданные параметры
baseline_ctr = 0.05  # базовый CTR
minimum_detectable_effect = 0.02  # минимально обнаружимый эффект
alpha = 0.05  # уровень значимости
power = 0.8  # мощность теста

# Расчет стандартного отклонения для Z-теста на равенство долей
pooled_prob = (baseline_ctr + baseline_ctr + minimum_detectable_effect) / 2
std_dev = math.sqrt(2 * pooled_prob * (1 - pooled_prob))

# Расчет Z-значения для уровня значимости alpha
from scipy.stats import norm
Z_alpha = norm.ppf(1 - alpha/2)

# Расчет объема выборки для каждой группы
n = (2 * (std_dev * Z_alpha + std_dev * norm.ppf(power))**2) / minimum_detectable_effect**2

print("Объем выборки для каждой группы в сплит-тесте:", math.ceil(n))


Объем выборки для каждой группы в сплит-тесте: 4427
